# Metrics manipulattion

Key resources
* https://docs.ragas.io/en/stable/concepts/metrics/index.html
* https://github.com/explodinggradients/ragas/tree/main/src/ragas/metrics (prompts and logic)
* https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/ (see the 'under the hood' section)

## prep

In [13]:
# !pip install -U openai

In [14]:
# !pip install openai
# # !pip uninstall -y openai

In [15]:
# import openai
# openai.__version__

In [16]:
# !pip install ragas

In [17]:
from copy import deepcopy
import os
import openai

In [18]:
from dotenv import load_dotenv
load_dotenv()

openaikey = os.getenv('OPENAI_API_KEY')
openai.api_key = openaikey

## Alter inputs to understand metrics
* https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/

## Eval single doc

In [19]:
import langchain

In [20]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [21]:
## set up and run the retrieval chain.
# - for one doc chunked.

# load the Wikipedia page and create index
loader = WebBaseLoader("https://en.wikipedia.org/wiki/New_York_City")
index = VectorstoreIndexCreator().from_loaders([loader])

# create the QA chain
llm = ChatOpenAI()
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=index.vectorstore.as_retriever(), return_source_documents=True
)

# testing it out
question = "How did New York City get its name?"
result = qa_chain({"query": question})
result["result"]

'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.'

In [22]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [23]:
results_orig = deepcopy(result)

In [24]:
## Prep the eval chain.
#
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.langchain import RagasEvaluatorChain

# make eval chains
eval_chains = {m.name: RagasEvaluatorChain(metric=m) for m in [faithfulness, answer_relevancy, context_precision] } # context_recall

/home/cataluna84/anaconda3/envs/generative-ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# Run each eval separately.
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")

faithfulness_score: 1.0
answer_relevancy_score: 0.9111665779493906
context_precision_score: 0.9999999999


## faithfullness (answer to the context)
* answer is regarded as faithful if all the claims that are made in the answer can be inferred from the given context.
* One prompt to extract statements (sentences?) and a second prompt to judge groundedness.

In [26]:
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

1.0

In [27]:
result = deepcopy(results_orig)

In [28]:
## Adding eroneous contexts/docs should NOT affect faithfulness.
# - sonce all the answers are still available in the context docs.

In [30]:
# Add four bad docs to the end of the results list.
myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'].extend([myd, myd, myd, myd])
result['source_documents']

[Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New Amsterdam (Dutch: Nieuw Amsterdam) in 1626 and was chartered as a city in 1653. The city came under British control in 1664 and was renamed New York after King Charles II granted the lands to his brother, the Duke of York.[17] The city was temporarily regained by the Dutch in July 1673

In [31]:
# This should have no effect on faithfulness, it's just extra, unused context.
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

1.0

In [32]:
## removing all the relevant docs should kill faithfulness.
#
result['source_documents'] = result['source_documents'][4:]
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/

In [33]:
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

0.6666666666666666

## answer_relevancy
* measures the relevancy of the answer to the question.
* gens questions from answer then measures similarity of these generate questions to original question.
* does not seem to work as expected. When kill answer and context-docs still getting 0.7 score.

In [34]:
result = deepcopy(results_orig)

In [35]:
RagasEvaluatorChain(metric=answer_relevancy)(result)['answer_relevancy_score']

0.9111665779493906

In [36]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [37]:
# result['result']
result

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of M

In [38]:
# result['result']  = result['result'] *4
# result['result']
# RagasEvaluatorChain(metric=answer_relevancy)(result)['answer_relevancy_score']

In [39]:
# Should kill this metric.
result['result'] = 'dog ' * 50
result['result'] 

'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog '

In [40]:
# Seems like this should be much much lower.
RagasEvaluatorChain(metric=answer_relevancy)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New 

In [41]:
## Alter the context to see what that does now that answer way wrong.
# - the prompt for this measure uses the context and the answer to generate the question.

# Add four bad docs to the end of the results list.
myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'] = [myd, myd, myd, myd]
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/

In [42]:
RagasEvaluatorChain(metric=answer_relevancy)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document

## context precision.
* https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html
* Search results order is key to this metric. Must have the answers in the top few context chunks to get a good score.
* So this retreival metric is all about returning good well ranked results.

In [43]:
result = deepcopy(results_orig)

In [44]:
RagasEvaluatorChain(metric=context_precision)(result) # ['context_precision_score']

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of M

In [45]:
# Add bad docs at end of results list. Should have little or no effect on context precision.
# - this metric is all about search results rank.

myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'].extend([myd, myd, myd, myd])

In [46]:
result['source_documents']

[Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New Amsterdam (Dutch: Nieuw Amsterdam) in 1626 and was chartered as a city in 1653. The city came under British control in 1664 and was renamed New York after King Charles II granted the lands to his brother, the Duke of York.[17] The city was temporarily regained by the Dutch in July 1673

In [47]:
RagasEvaluatorChain(metric=context_precision)(result)['context_precision_score']

0.9999999999

In [48]:
## reverse this list.
# - this should kill this metric.

result['source_documents'] = result['source_documents'][::-1]

In [49]:
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/

In [50]:
# RagasEvaluatorChain(metric=context_relevancy)(result)['context_relevancy_score']

## context_recall
* Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth.
* It is computed based on the ground truth and the retrieved context
* values range between 0 and 1, with higher values indicating better performance

In [51]:
result = deepcopy(results_orig)
result

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of M

In [52]:
# Add four bad docs to the end of the results list.
myd = deepcopy(result['source_documents'][0])
# myd.page_content = 'dog '*50
# result['source_documents'].extend([myd, myd, myd, myd])
# result['source_documents']

# Adding the ground_truths key for the context_recall evaluation
result['ground_truths'] = result['result']

result

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of M

In [53]:
context_recall_chain = RagasEvaluatorChain(metric=context_recall) #['context_recall_score']

context_recall_chain

RagasEvaluatorChain(metric=ContextRecall(batch_size=15, llm=OpenAI(model='gpt-3.5-turbo-16k', _api_key_env_var='OPENAI_API_KEY'), name='context_recall', evaluation_mode=<EvaluationMode.qcg: 7>))

In [54]:
context_recall_chain(result)

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after King Charles II granted the lands to his brother, the Duke of York. It was previously known as New Amsterdam when it was founded by Dutch colonists in 1626. The city came under British control in 1664 and was renamed New York.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of M

In [55]:
result.keys()

dict_keys(['query', 'result', 'source_documents', 'ground_truths'])

In [56]:
# result['result']  = result['result'] *4
# result['result']
# RagasEvaluatorChain(metric=answer_relevancy)(result)['answer_relevancy_score']

In [57]:
# Should kill this metric.
result['result'] = 'dog ' * 50
result['result'] 

'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog '

In [58]:
# Evaluate the results using the RagasEvaluatorChain with the context_recall metric.
# RagasEvaluatorChain: Likely a class or function for evaluating text generation results.
# metric=context_recall: Specifies the metric to use for evaluation, focusing on recall of relevant context.
# result: The output or results to be evaluated, likely a text string or generated text.


# Seems like this should be much much lower.
RagasEvaluatorChain(metric=context_recall)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[32] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[33]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New 

In [59]:
## Alter the context to see what that does now that answer way wrong.
# - the prompt for this measure uses the context and the answer to generate the question.

# Add four bad docs to the end of the results list.
myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'] = [myd, myd, myd, myd]
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/

In [60]:
RagasEvaluatorChain(metric=context_recall)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document